<a href="https://colab.research.google.com/github/MaiKhoa0101/MachineLearning/blob/main/CODE_Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall mediapipe -y
!pip cache purge
!pip install mediapipe==0.10.18
!python --version


Found existing installation: mediapipe 0.10.32
Uninstalling mediapipe-0.10.32:
  Successfully uninstalled mediapipe-0.10.32
Files removed: 24
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.1 MB/s eta 0:00:00
  Attemp

Python 3.12.12


In [ ]:
!pip install mediapipe

In [ ]:
import mediapipe as mp

# Kiểm tra xem có module solutions không
print(hasattr(mp, 'solutions'))  # Phải trả về True

# Nếu True, test tiếp:
print(mp.solutions.hands)

False


AttributeError: module 'mediapipe' has no attribute 'solutions'

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import json
from math import sqrt, atan2, degrees, acos, asin, atan
from pathlib import Path
from typing import Dict, Optional, Tuple

"""
SIGN LANGUAGE DETECTOR V9 - COMPLETE WITH WRIST ORIENTATION
============================================================
✅ Thêm: Wrist orientation detection (pitch, yaw, roll)
✅ Full body tracking hoàn chỉnh

Author: Claude
Date: 2026-01-09
"""

class RigConfiguration:
    """Cấu hình ánh xạ giữa góc vật lý và trục xương của Model 3D"""

    def __init__(self):
        # UPPER ARM
        self.UPPERARM_L = {
            "y": {"axis": "y", "scale": 1.0, "offset": 0},
            "z": {"axis": "z", "scale": 1.0, "offset": 0},
            "x": {"axis": "x", "scale": -1.0, "offset": 0}
        }
        self.UPPERARM_R = {
            "y": {"axis": "y", "scale": 1.0, "offset": 0},
            "z": {"axis": "z", "scale": 1.0, "offset": 0},
            "x": {"axis": "x", "scale": -1.0, "offset": 0}
        }

        # LOWER ARM
        self.LOWERARM_L = {
            "bend": {"axis": "z", "scale": -1.0, "offset": 0},
            "rotation": {"axis": "x", "scale": 1.0, "offset": 0}
        }
        self.LOWERARM_R = {
            "bend": {"axis": "z", "scale": -1.0, "offset": 0},
            "rotation": {"axis": "x", "scale": -1.0, "offset": 0}
        }

        # WRIST/HAND - ✅ CẬP NHẬT ĐẦY ĐỦ 3-DOF
        self.HAND_L = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},   # Cúi/ngửa cổ tay
            "yaw": {"axis": "y", "scale": 0.5, "offset": 0},     # Quay trái/phải
            "roll": {"axis": "x", "scale": 1.0, "offset": 0}     # Xoay (pronation/supination)
        }
        self.HAND_R = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw": {"axis": "y", "scale": 0.5, "offset": 0},
            "roll": {"axis": "x", "scale": 1.0, "offset": 0}
        }

        # FINGERS
        self.FINGER_CONFIG = {"bend": {"axis": "z", "scale": 1.0, "offset": 0}}

        # HEAD & NECK
        self.HEAD = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw": {"axis": "x", "scale": 1.0, "offset": 0},
            "roll": {"axis": "y", "scale": 1.0, "offset": 0}
        }
        self.NECK = {
            "pitch": {"axis": "z", "scale": 0.5, "offset": 0},
            "yaw": {"axis": "x", "scale": 0.5, "offset": 0}
        }

        # SPINE
        self.SPINE = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw": {"axis": "x", "scale": 1.0, "offset": 0}
        }

    def apply_mapping(self, physical_angles: Dict, config: Dict) -> Dict:
        """Áp dụng configuration để chuyển góc vật lý thành output Model"""
        result = {"x": 0, "y": 0, "z": 0}
        for angle_type, angle_value in physical_angles.items():
            if angle_type in config:
                mapping = config[angle_type]
                axis = mapping["axis"]
                scale = mapping["scale"]
                offset = mapping["offset"]
                result[axis] = round(angle_value * scale + offset, 2)
        return result

# ============================================================================
# PHYSICAL ANGLE CALCULATOR
# ============================================================================

class PhysicalAngleCalculator:
    """Tính góc vật lý thực tế từ MediaPipe landmarks"""

    @staticmethod
    def calculate_elbow_bend(shoulder, elbow, wrist) -> float:
        """Tính góc gập khuỷu tay (Law of Cosines)"""
        a = np.linalg.norm([elbow.x - shoulder.x, elbow.y - shoulder.y, elbow.z - shoulder.z])
        b = np.linalg.norm([wrist.x - elbow.x, wrist.y - elbow.y, wrist.z - elbow.z])
        c = np.linalg.norm([wrist.x - shoulder.x, wrist.y - shoulder.y, wrist.z - shoulder.z])

        cos_angle = (a*a + b*b - c*c) / (2 * a * b + 1e-6)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        angle = degrees(acos(cos_angle))
        return angle

    @staticmethod
    def calculate_forearm_rotation(shoulder, elbow, wrist) -> float:
        """Tính góc xoay cẳng tay"""
        upper_arm = np.array([
            elbow.x - shoulder.x,
            elbow.y - shoulder.y,
            elbow.z - shoulder.z
        ])
        upper_arm = upper_arm / (np.linalg.norm(upper_arm) + 1e-6)

        forearm = np.array([
            wrist.x - elbow.x,
            wrist.y - elbow.y,
            wrist.z - elbow.z
        ])
        forearm = forearm / (np.linalg.norm(forearm) + 1e-6)

        projection = np.dot(forearm, upper_arm) * upper_arm
        perpendicular = forearm - projection
        perp_length = np.linalg.norm(perpendicular) + 1e-6
        perpendicular = perpendicular / perp_length

        reference = np.array([0, 1, 0])
        ref_proj = reference - np.dot(reference, upper_arm) * upper_arm
        ref_length = np.linalg.norm(ref_proj) + 1e-6
        ref_proj = ref_proj / ref_length

        cos_angle = np.clip(np.dot(perpendicular, ref_proj), -1.0, 1.0)
        rotation_angle = degrees(acos(cos_angle))

        cross = np.cross(ref_proj, perpendicular)
        if np.dot(cross, upper_arm) < 0:
            rotation_angle = -rotation_angle

        return round(rotation_angle, 2)

    @staticmethod
    def calculate_wrist_orientation(shoulder, elbow, wrist, hand_landmarks) -> Dict[str, float]:
        """
        ✅ TÍNH GÓC CỔ TAY - SIMPLE Y-AXIS ANGLE

        Chỉ dùng Y component để tính flexion/extension
        """
        if hand_landmarks is None:
            return {"pitch": 0.0, "yaw": 0.0, "roll": 0.0}

        wrist_lm = hand_landmarks.landmark[0]     # Wrist
        middle_mcp = hand_landmarks.landmark[9]   # Middle base

        # Chỉ quan tâm đến Y difference (vertical)
        dy = middle_mcp.y - wrist_lm.y

        # Tính góc từ Y component
        # Trong MediaPipe hand: Y tăng = xuống dưới
        # dy > 0 = hand pointing down (flexion)
        # dy < 0 = hand pointing up (extension)

        # Scale xuống để có góc hợp lý (dựa vào thực nghiệm)
        pitch = np.clip(dy * 60, -40, 40)

        # Clamp
        pitch = np.clip(pitch, -45, 90)

        return {
            "pitch": round(pitch, 2),
            "yaw": 0.0,
            "roll": 0.0
        }

    @staticmethod
    def calculate_arm_orientation(shoulder, elbow, wrist=None):
        """Tính orientation cánh tay bằng spherical coordinates"""

        dx = elbow.x - shoulder.x
        dy = elbow.y - shoulder.y
        dz = elbow.z - shoulder.z

        length = sqrt(dx*dx + dy*dy + dz*dz) + 1e-6
        dx /= length
        dy /= length
        dz /= length

        # Yaw: xoay ngang (trái/phải)
        yaw = degrees(atan2(dx, dz))

        # Pitch: nâng/hạ tay
        pitch = degrees(atan2(-dy, sqrt(dx*dx + dz*dz)))

        # Roll: xoay quanh trục arm
        roll = 0.0

        if wrist is not None:
            wx = wrist.x - elbow.x
            wy = wrist.y - elbow.y
            wz = wrist.z - elbow.z

            arm_axis = np.array([dx, dy, dz])
            wrist_vec = np.array([wx, wy, wz])

            projection = np.dot(wrist_vec, arm_axis) * arm_axis
            perpendicular = wrist_vec - projection

            perp_length = np.linalg.norm(perpendicular) + 1e-6
            perpendicular /= perp_length

            ref = np.array([0, -1, 0])
            ref_proj = ref - np.dot(ref, arm_axis) * arm_axis
            ref_proj /= np.linalg.norm(ref_proj) + 1e-6

            cos_angle = np.clip(np.dot(perpendicular, ref_proj), -1.0, 1.0)
            roll = degrees(acos(cos_angle))

            if np.dot(np.cross(ref_proj, perpendicular), arm_axis) < 0:
                roll = -roll

        return {
            "x": round(roll, 2),
            "y": round(yaw, 2),
            "z": round(pitch, 2)
        }


    @staticmethod
    def calculate_finger_bend(landmarks, finger_ids) -> float:
        """
        Tính góc gập tại khớp PIP (ổn định hơn)
        Dùng vector dot product thay vì Law of Cosines
        """

        # Lấy 3 điểm chính: MCP - PIP - DIP
        p1 = landmarks[finger_ids[0]]  # MCP
        p2 = landmarks[finger_ids[1]]  # PIP (khớp chính)
        p3 = landmarks[finger_ids[2]]  # DIP

        v1 = np.array([p1.x - p2.x, p1.y - p2.y, p1.z - p2.z])
        v2 = np.array([p3.x - p2.x, p3.y - p2.y, p3.z - p2.z])

        v1 = v1 / (np.linalg.norm(v1) + 1e-6)
        v2 = v2 / (np.linalg.norm(v2) + 1e-6)

        cos_angle = np.clip(np.dot(v1, v2), -1.0, 1.0)
        angle = degrees(acos(cos_angle))

        # Chuyển về bend (0 = thẳng, tăng dần khi gập)
        bend = 180 - angle

        return round(bend, 2)


    # fix
    # def calculate_finger_bend(landmarks, finger_ids) -> float:
    #     """Tính góc gập trung bình của ngón tay"""
    #     total_angle = 0
    #     count = 0

    #     for i in range(1, len(finger_ids)-1):
    #         p1 = landmarks[finger_ids[i-1]]
    #         p2 = landmarks[finger_ids[i]]
    #         p3 = landmarks[finger_ids[i+1]]

    #         a = np.linalg.norm([p2.x - p1.x, p2.y - p1.y, p2.z - p1.z])
    #         b = np.linalg.norm([p3.x - p2.x, p3.y - p2.y, p3.z - p2.z])
    #         c = np.linalg.norm([p3.x - p1.x, p3.y - p1.y, p3.z - p1.z])

    #         cos_angle = (a*a + b*b - c*c) / (2 * a * b + 1e-6)
    #         cos_angle = np.clip(cos_angle, -1.0, 1.0)

    #         angle = degrees(acos(cos_angle))
    #         total_angle += (180 - angle)
    #         count += 1

    #     avg_angle = total_angle / count if count > 0 else 0
    #     return round(avg_angle, 2)

# ============================================================================
# GESTURE DETECTOR
# ============================================================================

class GestureDetector:
    """Detector sử dụng Physical Calculation + Rig Configuration"""

    def __init__(self):
        self.calc = PhysicalAngleCalculator()
        self.rig = RigConfiguration()

    def detect_arm(self, pose_lm, hand_landmarks, side='left') -> Dict:
        """Phát hiện và chuyển đổi góc cánh tay - ✅ ĐÃ CÓ WRIST ORIENTATION"""
        if side == 'left':
            shoulder, elbow, wrist = pose_lm[11], pose_lm[13], pose_lm[15]
            suffix = 'l'
            upperarm_config = self.rig.UPPERARM_L
            lowerarm_config = self.rig.LOWERARM_L
            hand_config = self.rig.HAND_L
        else:
            shoulder, elbow, wrist = pose_lm[12], pose_lm[14], pose_lm[16]
            suffix = 'r'
            upperarm_config = self.rig.UPPERARM_R
            lowerarm_config = self.rig.LOWERARM_R
            hand_config = self.rig.HAND_R

        # Calculate angles
        upperarm_angles = self.calc.calculate_arm_orientation(shoulder, elbow)

        if hand_landmarks is None:
            elbow_bend = 0
            forearm_rotation = 0
            wrist_angles = {"pitch": 0, "yaw": 0, "roll": 0}
        else:
            elbow_bend = self.calc.calculate_elbow_bend(shoulder, elbow, wrist)
            forearm_rotation = self.calc.calculate_forearm_rotation(shoulder, elbow, wrist)
            # ✅ FIXED: Truyền đầy đủ shoulder, elbow, wrist
            wrist_angles = self.calc.calculate_wrist_orientation(
                shoulder, elbow, wrist, hand_landmarks
            )
        # Apply rig configuration
        upperarm_result = self.rig.apply_mapping(upperarm_angles, upperarm_config)

        lowerarm_result = self.rig.apply_mapping({
            "bend": elbow_bend,
            "rotation": forearm_rotation
        }, lowerarm_config)

        # ✅ MAP WRIST ORIENTATION
        hand_result = self.rig.apply_mapping(wrist_angles, hand_config)

        output = {
            f"shoulder_{suffix}": f"shoulder_{suffix}(x=0, y=0, z=0)",
            f"upperarm_{suffix}": f"upperarm_{suffix}(x={upperarm_result['x']}, y={upperarm_result['y']}, z={upperarm_result['z']})",
            f"lowerarm_{suffix}": f"lowerarm_{suffix}(x={lowerarm_result['x']}, y={lowerarm_result['y']}, z={lowerarm_result['z']})",
            # ✅ WRIST ORIENTATION ĐẦY ĐỦ
            f"hand_{suffix}": f"hand_{suffix}(x={hand_result['x']}, y={hand_result['y']}, z={hand_result['z']})"
        }

        # Fingers
        if hand_landmarks is None:
            for finger in ['thumb', 'index', 'middle', 'ring', 'pinky']:
                output[f"{finger}_{suffix}"] = self._default_finger(finger, suffix)
        else:
            hand_lm = hand_landmarks.landmark

            finger_ids = {
                "thumb": [1, 2, 3, 4],
                "index": [5, 6, 7, 8],
                "middle": [9, 10, 11, 12],
                "ring": [13, 14, 15, 16],
                "pinky": [17, 18, 19, 20]
            }

            for fname, ids in finger_ids.items():
                bend = self.calc.calculate_finger_bend(hand_lm, ids)
                finger_result = self.rig.apply_mapping({"bend": bend}, self.rig.FINGER_CONFIG)
                output[f"{fname}_{suffix}"] = self._format_finger(fname, suffix, finger_result['z'])

        return output

    def detect_head_neck(self, pose_lm) -> Dict:
        """Phát hiện head & neck"""
        return {
            "head": "head(x=0, y=0, z=0)",
            "neck": "neck(x=0, y=0, z=0)"
        }

    def detect_spine(self, pose_lm) -> Dict:
        """Phát hiện spine"""
        return {
            "spine_01": "spine_01(x=0, y=0, z=0)",
            "spine_02": "spine_02(x=0, y=0, z=0)",
            "spine_03": "spine_03(x=0, y=0, z=0)"
        }

    def _default_finger(self, finger, suffix):
        if finger == 'thumb':
            return f"thumb_02_{suffix}(x=0, y=0, z=0); thumb_03_{suffix}(x=0, y=0, z=0)"
        else:
            return f"{finger}_01_{suffix}(x=0, y=0, z=0); {finger}_02_{suffix}(x=0, y=0, z=0); {finger}_03_{suffix}(x=0, y=0, z=0)"

    def _format_finger(self, finger, suffix, bend_angle):
        if finger == 'thumb':
            return f"thumb_02_{suffix}(x=0, y=0, z={bend_angle}); thumb_03_{suffix}(x=0, y=0, z={bend_angle})"
        else:
            return f"{finger}_01_{suffix}(x=0, y=0, z={bend_angle}); {finger}_02_{suffix}(x=0, y=0, z={bend_angle}); {finger}_03_{suffix}(x=0, y=0, z={bend_angle})"


# ============================================================================
# MAIN DETECTOR
# ============================================================================

class SignLanguageDetector:
    """Main detector với Motion Retargeting"""

    def __init__(self):
        self.mp_holistic = mp.solutions.holistic
        self.holistic = self.mp_holistic.Holistic(
            min_detection_confidence=0.3,
            min_tracking_confidence=0.3,
            model_complexity=1
        )
        self.detector = GestureDetector()
        print("✅ SignLanguageDetector V9 (WITH WRIST ORIENTATION) initialized")

    def process_frame(self, frame) -> Tuple[Optional[Dict], Optional[np.ndarray]]:
        """Process frame and return gesture data + skeleton image"""
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.holistic.process(img_rgb)

        if not results.pose_landmarks:
            return None, None

        # Draw skeleton
        skeleton_img = self.draw_skeleton(frame, results)

        pose_lm = results.pose_landmarks.landmark

        # Detect all body parts
        output = {}
        output.update(self.detector.detect_spine(pose_lm))
        output.update(self.detector.detect_head_neck(pose_lm))

        # Facial (default values)
        output.update({
            "jaw": "jaw(z=0)",
            "eyelid_l": "eyelid_l(z=0)",
            "eyelid_r": "eyelid_r(z=0)",
            "eyes": "eye_l(y=0); eye_r(y=0)",
            "eyebrows": "eyebrow_l(z=0); eyebrow_r(z=0)",
            "mouth_l": "mouth_l(x=0, y=0, z=0)",
            "mouth_r": "mouth_r(x=0, y=0, z=0)"
        })

        # Arms (with wrist orientation)
        output.update(self.detector.detect_arm(pose_lm, results.left_hand_landmarks, 'left'))
        output.update(self.detector.detect_arm(pose_lm, results.right_hand_landmarks, 'right'))

        return output, skeleton_img

    def draw_skeleton(self, frame, results):
        """Draw skeleton with wrist orientation indicators"""
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing_styles = mp.solutions.drawing_styles
        mp_holistic = mp.solutions.holistic

        annotated = frame.copy()
        h, w, _ = annotated.shape

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                annotated, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

        pose_lm = results.pose_landmarks.landmark if results.pose_landmarks else None

        # Draw hand landmarks with wrist orientation
        if results.right_hand_landmarks and pose_lm:
            mp_drawing.draw_landmarks(
                annotated, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

            # ✅ VẼ WRIST ORIENTATION CHO TAY PHẢI
            shoulder_r = pose_lm[12]  # Right shoulder
            elbow_r = pose_lm[14]     # Right elbow
            wrist_r = pose_lm[16]     # Right wrist

            wrist_angles = self.detector.calc.calculate_wrist_orientation(
                shoulder_r, elbow_r, wrist_r, results.right_hand_landmarks
            )
            self._draw_wrist_orientation(annotated, wrist_r, w, h, wrist_angles, "R")

        if results.left_hand_landmarks and pose_lm:
            mp_drawing.draw_landmarks(
                annotated, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

            # ✅ VẼ WRIST ORIENTATION CHO TAY TRÁI
            shoulder_l = pose_lm[11]  # Left shoulder
            elbow_l = pose_lm[13]     # Left elbow
            wrist_l = pose_lm[15]     # Left wrist

            wrist_angles = self.detector.calc.calculate_wrist_orientation(
                shoulder_l, elbow_l, wrist_l, results.left_hand_landmarks
            )
            self._draw_wrist_orientation(annotated, wrist_l, w, h, wrist_angles, "L")

        return annotated

    def _draw_wrist_orientation(self, img, wrist_pose, img_w, img_h, angles, side):
        """
        ✅ VẼ CHỈ BÁO WRIST ORIENTATION - FIXED

        Args:
            img: Image to draw on
            wrist_pose: Wrist landmark from pose
            img_w: Image width
            img_h: Image height
            angles: Dict với pitch, yaw, roll
            side: "L" hoặc "R"
        """
        cx = int(wrist_pose.x * img_w)
        cy = int(wrist_pose.y * img_h)

        if cx < 0 or cx >= img_w or cy < 0 or cy >= img_h:
            return

        # Vẽ box hiển thị angles
        box_width = 120
        box_height = 60
        box_x = cx - box_width // 2
        box_y = cy - 80

        # Đảm bảo box không vượt khỏi frame
        box_x = max(0, min(box_x, img_w - box_width))
        box_y = max(0, min(box_y, img_h - box_height))

        # Background
        cv2.rectangle(img,
                    (box_x, box_y),
                    (box_x + box_width, box_y + box_height),
                    (0, 0, 0), -1)
        cv2.rectangle(img,
                    (box_x, box_y),
                    (box_x + box_width, box_y + box_height),
                    (0, 255, 255), 2)

        # Text
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.4
        thickness = 1

        # Title
        cv2.putText(img, f"{side}_Wrist",
                  (box_x + 5, box_y + 15),
                  font, font_scale, (0, 255, 255), thickness)

        # Angles
        cv2.putText(img, f"P:{angles['pitch']:6.1f}",
                  (box_x + 5, box_y + 30),
                  font, font_scale, (255, 255, 255), thickness)
        cv2.putText(img, f"Y:{angles['yaw']:6.1f}",
                  (box_x + 5, box_y + 45),
                  font, font_scale, (255, 255, 255), thickness)
        cv2.putText(img, f"R:{angles['roll']:6.1f}",
                  (box_x + 70, box_y + 30),
                  font, font_scale, (255, 255, 255), thickness)

    def close(self):
        self.holistic.close()

# ============================================================================
# VIDEO PROCESSING
# ============================================================================

def process_video(video_path, output_json="gestures_v9.json", output_dir="frames_v9", frames_per_minute=1):
    """Process video with V9 Complete (with wrist)"""

    Path(output_dir).mkdir(exist_ok=True)
    frames_folder = Path(output_dir) / "original_frames"
    skeleton_folder = Path(output_dir) / "skeleton_frames"
    frames_folder.mkdir(exist_ok=True)
    skeleton_folder.mkdir(exist_ok=True)

    detector = SignLanguageDetector()
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"❌ Cannot open video: {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_frames / fps if fps > 0 else 0
    frame_interval = int(fps*0.3 / frames_per_minute) if fps > 0 else 1

    results = []
    frame_id = 0
    detected_count = 0

    print(f"\n{'='*70}")
    print(f"🎬 VIDEO: {video_path}")
    print(f"📊 FPS: {fps:.2f} | Duration: {duration_sec:.1f}s")
    print(f"⏱️  Sampling: 1 frame every {frame_interval} frames")
    print(f"📁 Output: {output_dir}/")
    print(f"✅ V9: COMPLETE WITH WRIST ORIENTATION")
    print(f"{'='*70}\n")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_id % frame_interval == 0:
            timestamp = frame_id / fps if fps > 0 else frame_id
            gesture_data, skeleton_img = detector.process_frame(frame)

            if gesture_data and skeleton_img is not None:
                frame_filename = f"frame_{frame_id:06d}_t{timestamp:.2f}s.jpg"
                skeleton_filename = f"skeleton_{frame_id:06d}_t{timestamp:.2f}s.jpg"

                cv2.imwrite(str(frames_folder / frame_filename), frame)
                cv2.imwrite(str(skeleton_folder / skeleton_filename), skeleton_img)

                results.append({
                    "frame": frame_id,
                    "timestamp": round(timestamp, 3),
                    "original_image": str(frames_folder / frame_filename),
                    "skeleton_image": str(skeleton_folder / skeleton_filename),
                    "gestures": gesture_data
                })
                detected_count += 1
                print(f"  ✅ Frame {frame_id:6d} at {timestamp:7.2f}s - Detected #{detected_count}")

        frame_id += 1

        if frame_id % 300 == 0:
            progress = (frame_id / total_frames * 100) if total_frames > 0 else 0
            print(f"  ⏳ Progress: {frame_id}/{total_frames} ({progress:.1f}%)")

    cap.release()
    detector.close()

    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\n{'='*70}")
    print(f"✅ COMPLETED!")
    print(f"📦 Detected: {detected_count} frames")
    print(f"💾 JSON: {output_json}")
    print(f"🖼️  Frames: {frames_folder}/ ({detected_count} images)")
    print(f"🦴 Skeleton: {skeleton_folder}/ ({detected_count} images)")
    print(f"{'='*70}\n")

# ============================================================================
# MAIN
# ============================================================================

if __name__ == "__main__":
    import sys
    import os

    VIDEO_PATH = "D0002.webm"
    OUTPUT_JSON = "gestures_v9_complete.json"
    OUTPUT_DIR = "frames_v9_complete"
    FRAMES_PER_MINUTE = 5

    if not os.path.exists(VIDEO_PATH):
        print(f"\n❌ ERROR: Video file not found: {VIDEO_PATH}")
        sys.exit(1)

    print(f"\n🚀 Starting Sign Language Detection V9 (COMPLETE)")
    print(f"✅ NEW: Wrist orientation detection (pitch, yaw, roll)")
    print(f"✅ Full body tracking hoàn chỉnh")

    process_video(VIDEO_PATH, OUTPUT_JSON, OUTPUT_DIR, FRAMES_PER_MINUTE)

    print("\n💡 V9 COMPLETE FEATURES:")
    print("   ✅ Upper arm: 3-DOF (pitch, yaw, roll)")
    print("   ✅ Lower arm: bend + rotation")
    print("   ✅ Wrist: 3-DOF (pitch, yaw, roll) ← MỚI!")
    print("   ✅ Fingers: individual bend angles")
    print("   ✅ Full body: head, neck, spine")
    print()

In [ ]:
!pip install mediapipe opencv-python-headless numpy -q
!pip install flask flask-cors pyngrok

In [ ]:
pip uninstall mediapipe -y
pip install mediapipe==0.10.14

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import json
import os
import base64
from IPython.display import display, HTML, clear_output
from math import sqrt, atan2, degrees, acos
from pathlib import Path
from typing import Dict, Optional, Tuple
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf

# Setup ngrok
conf.get_default().auth_token = "3721EzVULUpY9QkshdRLFveW6LV_2hLJT8sRukP7E4hw6zz6Y"

class RigConfiguration:
    def __init__(self):
        self.UPPERARM_L = {
            "y": {"axis": "y", "scale":  1.0, "offset": 0},
            "z": {"axis": "z", "scale":  1.0, "offset": 0},
            "x": {"axis": "x", "scale": -1.0, "offset": 0}
        }
        self.UPPERARM_R = {
            "y": {"axis": "y", "scale":  1.0, "offset": 0},
            "z": {"axis": "z", "scale":  1.0, "offset": 0},
            "x": {"axis": "x", "scale": -1.0, "offset": 0}
        }
        self.LOWERARM_L = {
            "bend":     {"axis": "z", "scale": -1.0, "offset": 0},
            "rotation": {"axis": "x", "scale":  1.0, "offset": 0}
        }
        self.LOWERARM_R = {
            "bend":     {"axis": "z", "scale": -1.0, "offset": 0},
            "rotation": {"axis": "x", "scale": -1.0, "offset": 0}
        }
        self.HAND_L = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw":   {"axis": "y", "scale": 0.5, "offset": 0},
            "roll":  {"axis": "x", "scale": 1.0, "offset": 0}
        }
        self.HAND_R = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw":   {"axis": "y", "scale": 0.5, "offset": 0},
            "roll":  {"axis": "x", "scale": 1.0, "offset": 0}
        }
        self.FINGER_CONFIG = {"bend": {"axis": "z", "scale": 1.0, "offset": 0}}
        self.HEAD  = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw":   {"axis": "x", "scale": 1.0, "offset": 0},
            "roll":  {"axis": "y", "scale": 1.0, "offset": 0}
        }
        self.NECK  = {
            "pitch": {"axis": "z", "scale": 0.5, "offset": 0},
            "yaw":   {"axis": "x", "scale": 0.5, "offset": 0}
        }
        self.SPINE = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw":   {"axis": "x", "scale": 1.0, "offset": 0}
        }

    def apply_mapping(self, physical_angles: Dict, config: Dict) -> Dict:
        result = {"x": 0, "y": 0, "z": 0}
        for angle_type, angle_value in physical_angles.items():
            if angle_type in config:
                m = config[angle_type]
                result[m["axis"]] = round(angle_value * m["scale"] + m["offset"], 2)
        return result


class PhysicalAngleCalculator:

    @staticmethod
    def calculate_elbow_bend(shoulder, elbow, wrist) -> float:
        a = np.linalg.norm([elbow.x-shoulder.x, elbow.y-shoulder.y, elbow.z-shoulder.z])
        b = np.linalg.norm([wrist.x-elbow.x,    wrist.y-elbow.y,    wrist.z-elbow.z])
        c = np.linalg.norm([wrist.x-shoulder.x,  wrist.y-shoulder.y,  wrist.z-shoulder.z])
        cos_a = np.clip((a*a + b*b - c*c) / (2*a*b + 1e-6), -1.0, 1.0)
        return degrees(acos(cos_a))

    @staticmethod
    def calculate_forearm_rotation(shoulder, elbow, wrist) -> float:
        upper = np.array([elbow.x-shoulder.x, elbow.y-shoulder.y, elbow.z-shoulder.z])
        upper /= np.linalg.norm(upper) + 1e-6
        fore = np.array([wrist.x-elbow.x, wrist.y-elbow.y, wrist.z-elbow.z])
        fore /= np.linalg.norm(fore) + 1e-6
        perp = fore - np.dot(fore, upper) * upper
        perp /= np.linalg.norm(perp) + 1e-6
        ref = np.array([0, 1, 0])
        ref_p = ref - np.dot(ref, upper) * upper
        ref_p /= np.linalg.norm(ref_p) + 1e-6
        cos_a = np.clip(np.dot(perp, ref_p), -1.0, 1.0)
        rot = degrees(acos(cos_a))
        if np.dot(np.cross(ref_p, perp), upper) < 0:
            rot = -rot
        return round(rot, 2)

    @staticmethod
    def calculate_wrist_orientation(shoulder, elbow, wrist, hand_landmarks) -> Dict:
        if hand_landmarks is None:
            return {"pitch": 0.0, "yaw": 0.0, "roll": 0.0}
        wrist_lm   = hand_landmarks.landmark[0]
        middle_mcp = hand_landmarks.landmark[9]
        dy = middle_mcp.y - wrist_lm.y
        pitch = float(np.clip(dy * 60, -45, 90))
        return {"pitch": round(pitch, 2), "yaw": 0.0, "roll": 0.0}

    @staticmethod
    def calculate_arm_orientation(shoulder, elbow, wrist=None) -> Dict:
        dx = elbow.x - shoulder.x
        dy = elbow.y - shoulder.y
        dz = elbow.z - shoulder.z
        L = sqrt(dx*dx + dy*dy + dz*dz) + 1e-6
        dx /= L; dy /= L; dz /= L
        yaw   = degrees(atan2(dx, dz))
        pitch = degrees(atan2(-dy, sqrt(dx*dx + dz*dz)))
        roll  = 0.0
        if wrist is not None:
            arm = np.array([dx, dy, dz])
            wv  = np.array([wrist.x-elbow.x, wrist.y-elbow.y, wrist.z-elbow.z])
            perp = wv - np.dot(wv, arm) * arm
            perp /= np.linalg.norm(perp) + 1e-6
            ref = np.array([0, -1, 0])
            ref_p = ref - np.dot(ref, arm) * arm
            ref_p /= np.linalg.norm(ref_p) + 1e-6
            cos_a = np.clip(np.dot(perp, ref_p), -1.0, 1.0)
            roll = degrees(acos(cos_a))
            if np.dot(np.cross(ref_p, perp), arm) < 0:
                roll = -roll
        return {"x": round(roll, 2), "y": round(yaw, 2), "z": round(pitch, 2)}

    @staticmethod
    def calculate_finger_bend(landmarks, finger_ids) -> float:
        p1 = landmarks[finger_ids[0]]
        p2 = landmarks[finger_ids[1]]
        p3 = landmarks[finger_ids[2]]
        v1 = np.array([p1.x-p2.x, p1.y-p2.y, p1.z-p2.z])
        v2 = np.array([p3.x-p2.x, p3.y-p2.y, p3.z-p2.z])
        v1 /= np.linalg.norm(v1) + 1e-6
        v2 /= np.linalg.norm(v2) + 1e-6
        return round(180 - degrees(acos(np.clip(np.dot(v1, v2), -1.0, 1.0))), 2)


# ============================================================================
# GESTURE DETECTOR
# ============================================================================
class GestureDetector:
    def __init__(self):
        self.calc = PhysicalAngleCalculator()
        self.rig  = RigConfiguration()

    def detect_arm(self, pose_lm, hand_landmarks, side='left') -> Dict:
        if side == 'left':
            shoulder, elbow, wrist = pose_lm[11], pose_lm[13], pose_lm[15]
            suffix = 'l'
            up_cfg, lo_cfg, h_cfg = self.rig.UPPERARM_L, self.rig.LOWERARM_L, self.rig.HAND_L
        else:
            shoulder, elbow, wrist = pose_lm[12], pose_lm[14], pose_lm[16]
            suffix = 'r'
            up_cfg, lo_cfg, h_cfg = self.rig.UPPERARM_R, self.rig.LOWERARM_R, self.rig.HAND_R

        upperarm_angles = self.calc.calculate_arm_orientation(shoulder, elbow)

        if hand_landmarks is None:
            elbow_bend = forearm_rotation = 0
            wrist_angles = {"pitch": 0, "yaw": 0, "roll": 0}
        else:
            elbow_bend       = self.calc.calculate_elbow_bend(shoulder, elbow, wrist)
            forearm_rotation = self.calc.calculate_forearm_rotation(shoulder, elbow, wrist)
            wrist_angles     = self.calc.calculate_wrist_orientation(shoulder, elbow, wrist, hand_landmarks)

        up  = self.rig.apply_mapping(upperarm_angles, up_cfg)
        lo  = self.rig.apply_mapping({"bend": elbow_bend, "rotation": forearm_rotation}, lo_cfg)
        hnd = self.rig.apply_mapping(wrist_angles, h_cfg)

        output = {
            f"shoulder_{suffix}": f"shoulder_{suffix}(x=0, y=0, z=0)",
            f"upperarm_{suffix}": f"upperarm_{suffix}(x={up['x']}, y={up['y']}, z={up['z']})",
            f"lowerarm_{suffix}": f"lowerarm_{suffix}(x={lo['x']}, y={lo['y']}, z={lo['z']})",
            f"hand_{suffix}":     f"hand_{suffix}(x={hnd['x']}, y={hnd['y']}, z={hnd['z']})"
        }

        if hand_landmarks is None:
            for f in ['thumb', 'index', 'middle', 'ring', 'pinky']:
                output[f"{f}_{suffix}"] = self._default_finger(f, suffix)
        else:
            lm = hand_landmarks.landmark
            FINGERS = {
                "thumb":  [1, 2, 3, 4],
                "index":  [5, 6, 7, 8],
                "middle": [9, 10, 11, 12],
                "ring":   [13, 14, 15, 16],
                "pinky":  [17, 18, 19, 20]
            }
            for fname, ids in FINGERS.items():
                bend = self.calc.calculate_finger_bend(lm, ids)
                fr   = self.rig.apply_mapping({"bend": bend}, self.rig.FINGER_CONFIG)
                output[f"{fname}_{suffix}"] = self._format_finger(fname, suffix, fr['z'])
        return output

    def detect_head_neck(self, pose_lm) -> Dict:
        return {"head": "head(x=0, y=0, z=0)", "neck": "neck(x=0, y=0, z=0)"}

    def detect_spine(self, pose_lm) -> Dict:
        return {
            "spine_01": "spine_01(x=0, y=0, z=0)",
            "spine_02": "spine_02(x=0, y=0, z=0)",
            "spine_03": "spine_03(x=0, y=0, z=0)"
        }

    def _default_finger(self, finger, suffix):
        if finger == 'thumb':
            return f"thumb_02_{suffix}(x=0, y=0, z=0); thumb_03_{suffix}(x=0, y=0, z=0)"
        return f"{finger}_01_{suffix}(x=0, y=0, z=0); {finger}_02_{suffix}(x=0, y=0, z=0); {finger}_03_{suffix}(x=0, y=0, z=0)"

    def _format_finger(self, finger, suffix, z):
        if finger == 'thumb':
            return f"thumb_02_{suffix}(x=0, y=0, z={z}); thumb_03_{suffix}(x=0, y=0, z={z})"
        return f"{finger}_01_{suffix}(x=0, y=0, z={z}); {finger}_02_{suffix}(x=0, y=0, z={z}); {finger}_03_{suffix}(x=0, y=0, z={z})"


# ============================================================================
# MAIN DETECTOR
# ============================================================================
class SignLanguageDetector:
    def __init__(self):
        mp_holistic = mp.solutions.holistic
        self.holistic = mp_holistic.Holistic(
            min_detection_confidence=0.3,
            min_tracking_confidence=0.3,
            model_complexity=1
        )
        self.detector = GestureDetector()
        self.calc     = PhysicalAngleCalculator()

    def process_frame(self, frame) -> Tuple[Optional[Dict], Optional[np.ndarray]]:
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.holistic.process(img_rgb)
        if not results.pose_landmarks:
            return None, None
        skeleton_img = self._draw_skeleton(frame, results)
        pose_lm = results.pose_landmarks.landmark
        output = {}
        output.update(self.detector.detect_spine(pose_lm))
        output.update(self.detector.detect_head_neck(pose_lm))
        output.update({
            "jaw":      "jaw(z=0)",
            "eyelid_l": "eyelid_l(z=0)",
            "eyelid_r": "eyelid_r(z=0)",
            "eyes":     "eye_l(y=0); eye_r(y=0)",
            "eyebrows": "eyebrow_l(z=0); eyebrow_r(z=0)",
            "mouth_l":  "mouth_l(x=0, y=0, z=0)",
            "mouth_r":  "mouth_r(x=0, y=0, z=0)"
        })
        output.update(self.detector.detect_arm(pose_lm, results.left_hand_landmarks,  'left'))
        output.update(self.detector.detect_arm(pose_lm, results.right_hand_landmarks, 'right'))
        return output, skeleton_img

    def _draw_skeleton(self, frame, results):
        mp_drawing        = mp.solutions.drawing_utils
        mp_drawing_styles = mp.solutions.drawing_styles
        mp_holistic       = mp.solutions.holistic
        annotated = frame.copy()
        h, w, _   = annotated.shape
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                annotated, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )
        pose_lm = results.pose_landmarks.landmark if results.pose_landmarks else None
        for hand_lm, s_idx, e_idx, w_idx, label in [
            (results.right_hand_landmarks, 12, 14, 16, 'R'),
            (results.left_hand_landmarks,  11, 13, 15, 'L')
        ]:
            if hand_lm and pose_lm:
                mp_drawing.draw_landmarks(
                    annotated, hand_lm, mp_holistic.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )
                angles = self.calc.calculate_wrist_orientation(
                    pose_lm[s_idx], pose_lm[e_idx], pose_lm[w_idx], hand_lm
                )
                self._draw_wrist_info(annotated, pose_lm[w_idx], w, h, angles, label)
        return annotated

    def _draw_wrist_info(self, img, wrist_pose, img_w, img_h, angles, side):
        cx = int(wrist_pose.x * img_w)
        cy = int(wrist_pose.y * img_h)
        if not (0 <= cx < img_w and 0 <= cy < img_h):
            return
        bw, bh = 120, 60
        bx = max(0, min(cx - bw//2, img_w - bw))
        by = max(0, min(cy - 80,    img_h - bh))
        cv2.rectangle(img, (bx, by), (bx+bw, by+bh), (0, 0, 0), -1)
        cv2.rectangle(img, (bx, by), (bx+bw, by+bh), (0, 255, 255), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, f"{side}_Wrist",            (bx+5, by+15), font, 0.4, (0, 255, 255), 1)
        cv2.putText(img, f"P:{angles['pitch']:6.1f}", (bx+5, by+30), font, 0.4, (255, 255, 255), 1)
        cv2.putText(img, f"Y:{angles['yaw']:6.1f}",   (bx+5, by+45), font, 0.4, (255, 255, 255), 1)
        cv2.putText(img, f"R:{angles['roll']:6.1f}",  (bx+70,by+30), font, 0.4, (255, 255, 255), 1)

    def close(self):
        self.holistic.close()

# ============================================================================
# FLASK API
# ============================================================================

app = Flask(__name__)
CORS(app)

detector = SignLanguageDetector()

@app.route("/", methods=["GET"])
def home():
    return jsonify({
        "status": "API running",
        "message": "Sign Language API ready"
    })

@app.route("/detect", methods=["POST"])
def detect():

    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]

    img_bytes = np.frombuffer(file.read(), np.uint8)
    frame = cv2.imdecode(img_bytes, cv2.IMREAD_COLOR)

    if frame is None:
        return jsonify({"error": "Invalid image"}), 400

    result, _ = detector.process_frame(frame)

    return jsonify({
        "status": "success",
        "data": result
    })

if __name__ == "__main__":
    public_url = ngrok.connect(8000)
    print(" Public URL:", public_url)

    app.run(host="0.0.0.0", port=8000)


 Public URL: NgrokTunnel: "https://lorriane-noncongregative-benson.ngrok-free.dev" -> "http://localhost:8000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Feb/2026 14:45:27] "GET /detect HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2026 14:45:28] "GET /detect HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2026 14:46:34] "GET /detect HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2026 14:46:43] "GET /detect HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2026 14:46:53] "GET /detect HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2026 14:46:57] "GET /detect HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2026 14:48:02] "GET /detect HTTP/1.1" 405 -
/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_fact

In [ ]:
# import cv2
# import mediapipe as mp
# import numpy as np
# import json
# import os
# from math import sqrt, degrees, acos, atan2, asin
# from typing import Dict, Optional, Tuple, List

# """
# SIGN LANGUAGE DETECTOR V6 - CUSTOM RIG MAPPING
# ==============================================
# Giải pháp: Tách biệt việc tính toán góc vật lý và việc map vào trục Model.
# Người dùng tự cấu hình trục cho từng bộ phận trong RIG_CONFIG.

# Author: Claude
# Date: 2026-01-03
# """

# # ============================================================================
# # ⚠️ CẤU HÌNH TRỤC MODEL CỦA BẠN (SỬA Ở ĐÂY)
# # ============================================================================
# # Quy ước:
# # 'axis': trục của model ('x', 'y', 'z')
# # 'scale': tỉ lệ và hướng (1.0 là cùng chiều, -1.0 là ngược chiều, 0.0 là khóa trục)
# # 'offset': góc cộng thêm (để chỉnh pose mặc định)

# RIG_CONFIG = {
#     # --- CÁNH TAY TRÁI ---
#     "left_arm": {
#         # Đưa tay ra trước/sau (Flexion/Extension) - Bạn nói Z=-90 là ra trước
#         "pitch": {"axis": "z", "scale": -1.0, "offset": 0},
#         # Dang tay sang ngang (Abduction) - Thường là trục Y hoặc X
#         "yaw":   {"axis": "y", "scale": 1.0,  "offset": 80}, # Offset 80 để tay xuôi
#         # Xoay trục tay (Twist)
#         "roll":  {"axis": "x", "scale": 1.0,  "offset": 0},
#     },
#     "left_forearm": {
#         # Gập khuỷu tay (Elbow Flexion) - Thường là trục X hoặc Z
#         "bend":  {"axis": "x", "scale": -1.0, "offset": 0}
#     },

#     # --- CÁNH TAY PHẢI (Bạn nói chia sẻ chung giá trị với trái) ---
#     "right_arm": {
#         # Đưa tay ra trước (Bạn nói giống tay trái: Z = -90)
#         "pitch": {"axis": "z", "scale": -1.0, "offset": 0},
#         # Dang tay (Thường ngược dấu tay trái nếu đối xứng, nhưng bạn check lại)
#         "yaw":   {"axis": "y", "scale": -1.0, "offset": -80},
#         "roll":  {"axis": "x", "scale": 1.0,  "offset": 0},
#     },
#     "right_forearm": {
#         "bend":  {"axis": "x", "scale": -1.0, "offset": 0}
#     },

#     # --- NGÓN TAY (Thường ngón tay gập quanh trục X hoặc Z) ---
#     # Cấu hình chung cho các ngón (có thể tách riêng nếu cần)
#     "fingers": {
#         "bend_axis": "z",   # Trục gập ngón tay (Sửa thành 'x' hoặc 'z' tùy model)
#         "bend_scale": 1.0,  # Hướng gập
#         "spread_axis": "y"  # Trục xòe ngón (thường ít dùng)
#     }
# }

# # ============================================================================
# # LOGIC TÍNH TOÁN VẬT LÝ (KHÔNG CẦN SỬA)
# # ============================================================================

# def calculate_3d_angle(a, b, c):
#     """Tính góc gập giữa 3 điểm trong không gian 3D"""
#     ba = a - b
#     bc = c - b
#     cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
#     angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
#     return angle

# def get_elevation_azimuth(p_start, p_end):
#     """
#     Tính góc nâng (Elevation - Pitch) và góc ngang (Azimuth - Yaw)
#     của một vector trong hệ tọa độ chuẩn (MediaPipe)
#     """
#     dx = p_end[0] - p_start[0]
#     dy = p_end[1] - p_start[1]
#     dz = p_end[2] - p_start[2]

#     # MediaPipe: Y xuống (+), X phải (+), Z tới (+)
#     # Chuyển sang hệ quy chiếu người quan sát để dễ hình dung:
#     # Up/Down (Pitch): Dựa vào DY
#     # Left/Right (Yaw): Dựa vào DX
#     # Front/Back: Dựa vào DZ

#     # Tính Pitch (Nâng hạ): Góc với mặt phẳng ngang (XZ)
#     # distance_horizontal = sqrt(dx*dx + dz*dz)
#     # pitch = degrees(atan2(dy, distance_horizontal)) # -90 (lên) đến 90 (xuống)

#     # Ở đây ta dùng vector đơn vị để map vào cấu hình
#     v = np.array([dx, dy, dz])
#     v = v / (np.linalg.norm(v) + 1e-6)

#     # Góc Pitch vật lý (Đưa tay ra trước/sau)
#     # Z âm là đưa ra trước (trong MP Z+ là xa camera, Z- là gần camera?)
#     # MP: Z value is depth. Negative is closer to camera.
#     # Vector cánh tay đưa ra trước: dz sẽ âm lớn.
#     pitch_physical = np.degrees(np.arctan2(v[2], v[1])) # Góc trong mặt phẳng YZ

#     # Góc Yaw vật lý (Dang tay sang ngang)
#     yaw_physical = np.degrees(np.arctan2(v[0], v[1]))   # Góc trong mặt phẳng XY

#     return pitch_physical, yaw_physical

# # ============================================================================
# # KINEMATIC SOLVER
# # ============================================================================

# class KinematicSolver:
#     def __init__(self):
#         pass

#     def apply_config(self, phys_val, config_key, part_config):
#         """Map giá trị vật lý vào trục model dựa trên RIG_CONFIG"""
#         cfg = part_config[config_key]
#         axis = cfg['axis']
#         scale = cfg['scale']
#         offset = cfg['offset']

#         mapped_val = (phys_val * scale) + offset
#         return axis, int(mapped_val)

#     def create_rot_string(self, name, mapping):
#         """Tạo chuỗi rotation từ dict mapping {axis: value}"""
#         x = mapping.get('x', 0)
#         y = mapping.get('y', 0)
#         z = mapping.get('z', 0)
#         return f"{name}(x={x}, y={y}, z={z})"

#     def solve_arm(self, landmarks, side='left'):
#         # 1. Lấy tọa độ landmarks
#         if side == 'left':
#             sh = np.array([landmarks[11].x, landmarks[11].y, landmarks[11].z])
#             el = np.array([landmarks[13].x, landmarks[13].y, landmarks[13].z])
#             wr = np.array([landmarks[15].x, landmarks[15].y, landmarks[15].z])
#             cfg_name = "left_arm"
#             fore_name = "left_forearm"
#             suffix = 'l'
#         else:
#             sh = np.array([landmarks[12].x, landmarks[12].y, landmarks[12].z])
#             el = np.array([landmarks[14].x, landmarks[14].y, landmarks[14].z])
#             wr = np.array([landmarks[16].x, landmarks[16].y, landmarks[16].z])
#             cfg_name = "right_arm"
#             fore_name = "right_forearm"
#             suffix = 'r'

#         # 2. Tính toán vật lý (Physical Calculation)

#         # A. Bắp tay (Upper Arm)
#         # Vector vai -> khuỷu
#         vec_upper = el - sh

#         # Góc đưa trước/sau (Pitch): Dựa trên trục Z của MP
#         # Khi tay xuôi (Y+), đưa ra trước (Z-).
#         # Ta cần tính độ mở góc so với trục dọc cơ thể.
#         # atan2(z, y): 0 là xuôi, -90 là đưa thẳng ra trước.
#         pitch_phys = np.degrees(np.arctan2(vec_upper[2], vec_upper[1]))

#         # Góc dang ngang (Yaw): Dựa trên trục X của MP
#         # atan2(x, y): 0 là xuôi, 90 là dang ngang.
#         yaw_phys = np.degrees(np.arctan2(vec_upper[0], vec_upper[1]))

#         # B. Cẳng tay (Forearm)
#         # Góc gập khuỷu tay (luôn dương, 180 là thẳng, 90 là vuông)
#         elbow_angle = calculate_3d_angle(sh, el, wr)
#         # Model thường tính góc gập từ 0 (thẳng) đến 90 (vuông) hoặc ngược lại
#         # Giả sử model của bạn: 0 là thẳng.
#         bend_phys = 180 - elbow_angle

#         # 3. Mapping vào Model (Dùng RIG_CONFIG)
#         u_map = {'x': 0, 'y': 0, 'z': 0}
#         l_map = {'x': 0, 'y': 0, 'z': 0}

#         # Map Upper Arm
#         p_axis, p_val = self.apply_config(pitch_phys, 'pitch', RIG_CONFIG[cfg_name])
#         y_axis, y_val = self.apply_config(yaw_phys, 'yaw', RIG_CONFIG[cfg_name])

#         # Cộng dồn giá trị vào trục (vì 1 trục có thể chịu tác động của cả pitch và yaw tùy model)
#         u_map[p_axis] += p_val
#         u_map[y_axis] += y_val

#         # Map Lower Arm
#         b_axis, b_val = self.apply_config(bend_phys, 'bend', RIG_CONFIG[fore_name])
#         l_map[b_axis] += b_val

#         # 4. Tạo output string
#         upper_str = self.create_rot_string(f"upperarm_{suffix}", u_map)
#         lower_str = self.create_rot_string(f"lowerarm_{suffix}", l_map)

#         return {
#             f"upperarm_{suffix}": upper_str,
#             f"lowerarm_{suffix}": lower_str,
#             f"shoulder_{suffix}": f"shoulder_{suffix}(x=0,y=0,z=0)", # Vai thường ít động
#             f"hand_{suffix}": f"hand_{suffix}(x=0,y=0,z=0)" # Tạm thời để 0
#         }

#     def solve_fingers(self, landmarks, side='left'):
#         # Logic gập ngón tay (tính góc gập và map vào trục cấu hình)
#         suffix = 'l' if side == 'left' else 'r'
#         fingers_out = {}

#         f_indices = {
#             'thumb': [1, 2, 3, 4], 'index': [5, 6, 7, 8],
#             'middle': [9, 10, 11, 12], 'ring': [13, 14, 15, 16],
#             'pinky': [17, 18, 19, 20]
#         }

#         cfg = RIG_CONFIG['fingers']
#         axis = cfg['bend_axis']
#         scale = cfg['bend_scale']

#         for name, ids in f_indices.items():
#             # Lấy landmarks
#             # Hand landmarks coordinates are relative to wrist usually in MP Hand model,
#             # but here we have holistic.

#             # Tính góc gập trung bình
#             # MCP bend
#             angle = calculate_3d_angle(
#                 np.array([landmarks[ids[0]].x, landmarks[ids[0]].y, landmarks[ids[0]].z]),
#                 np.array([landmarks[ids[1]].x, landmarks[ids[1]].y, landmarks[ids[1]].z]),
#                 np.array([landmarks[ids[2]].x, landmarks[ids[2]].y, landmarks[ids[2]].z])
#             )

#             # 180 là thẳng, 90 là gập.
#             # Model: 0 là thẳng, 90 là gập (thường vậy)
#             bend_val = int((180 - angle) * scale)

#             # Tạo chuỗi cho 3 đốt (giả sử gập đều)
#             # Bone names: name_01, name_02, name_03
#             f_str = ""
#             for i in range(1, 4):
#                 # Ngón cái thường chỉ có 2 đốt xoay chính hoặc đặt tên khác, nhưng theo format cũ của bạn:
#                 # thumb_01, thumb_02, thumb_03
#                 # Với ngón cái, đốt 1 (CMC) ít gập hơn
#                 val = bend_val
#                 if name == 'thumb' and i == 1: val = int(bend_val * 0.5)

#                 f_map = {'x': 0, 'y': 0, 'z': 0}
#                 f_map[axis] = val

#                 seg_str = self.create_rot_string(f"{name}_0{i}_{suffix}", f_map)
#                 f_str += seg_str + "; "

#             fingers_out[name] = f_str.strip("; ")

#         return fingers_out

# # ============================================================================
# # MAIN DETECTOR CLASS
# # ============================================================================

# class SignLanguageDetectorV6:
#     def __init__(self):
#         self.mp_holistic = mp.solutions.holistic
#         self.holistic = self.mp_holistic.Holistic(
#             min_detection_confidence=0.5,
#             min_tracking_confidence=0.5,
#             model_complexity=1
#         )
#         self.solver = KinematicSolver()
#         print("✅ SignLanguageDetector V6 (Custom Rig) initialized")

#     def process_frame(self, frame) -> Optional[Dict]:
#         img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         results = self.holistic.process(img_rgb)

#         if not results.pose_landmarks: return None
#         pose_lm = results.pose_landmarks.landmark

#         # Analyze Arms
#         left_arm = self.solver.solve_arm(pose_lm, 'left')
#         right_arm = self.solver.solve_arm(pose_lm, 'right')

#         # Analyze Fingers
#         left_fingers = {}
#         right_fingers = {}

#         if results.right_hand_landmarks: # MP right is person left
#             left_fingers = self.solver.solve_fingers(results.right_hand_landmarks.landmark, 'left')

#         if results.left_hand_landmarks:
#             right_fingers = self.solver.solve_fingers(results.left_hand_landmarks.landmark, 'right')

#         # Combine Output
#         output = {
#             "spine": {"spine_01": "spine_01(x=0,y=0,z=0)"}, # Default
#             "facial": {}, # Default
#             **left_arm,
#             **right_arm,
#             "left_hand_fingers": left_fingers, # Structure slightly adjusted for clarity, you can flatten if needed
#             "right_hand_fingers": right_fingers
#         }

#         # Merge fingers back to arm dict structure if needed by Android app
#         # (Assuming Android app expects fingers inside left_arm dict)
#         output["left_arm"]["fingers"] = left_fingers
#         output["right_arm"]["fingers"] = right_fingers

#         # Clean up flat keys if structure requires nesting
#         # Based on previous JSON: left_arm contains fingers.

#         final_output = {
#             "spine": {"spine_01": "spine_01(x=0,y=0,z=0)"},
#             "facial": {"jaw": "jaw(z=0)"}, # Minimal defaults
#             "left_arm": {
#                 **left_arm,
#                 "fingers": left_fingers
#             },
#             "right_arm": {
#                 **right_arm,
#                 "fingers": right_fingers
#             }
#         }

#         return final_output

#     def close(self):
#         self.holistic.close()

# # ============================================================================
# # RUNNER
# # ============================================================================

# def process_video(video_path, output_json, frames_per_minute=60):
#     detector = SignLanguageDetectorV6()
#     cap = cv2.VideoCapture(video_path)

#     if not cap.isOpened():
#         print("Error opening video")
#         return

#     fps = cap.get(cv2.CAP_PROP_FPS)
#     frame_interval = int((fps * 60) / frames_per_minute) if frames_per_minute > 0 else 1

#     results = []
#     frame_id = 0

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret: break

#         if frame_id % frame_interval == 0:
#             data = detector.process_frame(frame)
#             if data:
#                 results.append({
#                     "frame": frame_id,
#                     "timestamp": round(frame_id/fps, 3),
#                     "gestures": data
#                 })
#                 if frame_id % 100 == 0: print(f"Processed frame {frame_id}")

#         frame_id += 1

#     cap.release()
#     detector.close()

#     with open(output_json, 'w') as f:
#         json.dump(results, f, indent=2)
#     print(f"Done. Saved to {output_json}")

# if __name__ == "__main__":
#     VIDEO = "test_video.webm"
#     if os.path.exists(VIDEO):
#         process_video(VIDEO, "gestures_v6_custom.json")
#     else:
#         print("Video not found")

In [ ]:
# ## chuyển file sang HTTP
# import cv2
# import mediapipe as mp
# import numpy as np
# import json
# import os
# from math import sqrt, degrees, acos, atan2, asin
# from typing import Dict, Optional, Tuple, List

# """
# SIGN LANGUAGE DETECTOR V6 - CUSTOM RIG MAPPING
# ==============================================
# Giải pháp: Tách biệt việc tính toán góc vật lý và việc map vào trục Model.
# Người dùng tự cấu hình trục cho từng bộ phận trong RIG_CONFIG.

# Author: Claude
# Date: 2026-01-03
# """

# # ============================================================================
# # ⚠️ CẤU HÌNH TRỤC MODEL CỦA BẠN (SỬA Ở ĐÂY)
# # ============================================================================
# # Quy ước:
# # 'axis': trục của model ('x', 'y', 'z')
# # 'scale': tỉ lệ và hướng (1.0 là cùng chiều, -1.0 là ngược chiều, 0.0 là khóa trục)
# # 'offset': góc cộng thêm (để chỉnh pose mặc định)

# RIG_CONFIG = {
#     # --- CÁNH TAY TRÁI ---
#     "left_arm": {
#         # Đưa tay ra trước/sau (Flexion/Extension) - Bạn nói Z=-90 là ra trước
#         "pitch": {"axis": "z", "scale": -1.0, "offset": 0},
#         # Dang tay sang ngang (Abduction) - Thường là trục Y hoặc X
#         "yaw":   {"axis": "y", "scale": 1.0,  "offset": 80}, # Offset 80 để tay xuôi
#         # Xoay trục tay (Twist)
#         "roll":  {"axis": "x", "scale": 1.0,  "offset": 0},
#     },
#     "left_forearm": {
#         # Gập khuỷu tay (Elbow Flexion) - Thường là trục X hoặc Z
#         "bend":  {"axis": "x", "scale": -1.0, "offset": 0}
#     },

#     # --- CÁNH TAY PHẢI (Bạn nói chia sẻ chung giá trị với trái) ---
#     "right_arm": {
#         # Đưa tay ra trước (Bạn nói giống tay trái: Z = -90)
#         "pitch": {"axis": "z", "scale": -1.0, "offset": 0},
#         # Dang tay (Thường ngược dấu tay trái nếu đối xứng, nhưng bạn check lại)
#         "yaw":   {"axis": "y", "scale": -1.0, "offset": -80},
#         "roll":  {"axis": "x", "scale": 1.0,  "offset": 0},
#     },
#     "right_forearm": {
#         "bend":  {"axis": "x", "scale": -1.0, "offset": 0}
#     },

#     # --- NGÓN TAY (Thường ngón tay gập quanh trục X hoặc Z) ---
#     # Cấu hình chung cho các ngón (có thể tách riêng nếu cần)
#     "fingers": {
#         "bend_axis": "z",   # Trục gập ngón tay (Sửa thành 'x' hoặc 'z' tùy model)
#         "bend_scale": 1.0,  # Hướng gập
#         "spread_axis": "y"  # Trục xòe ngón (thường ít dùng)
#     }
# }

# # ============================================================================
# # LOGIC TÍNH TOÁN VẬT LÝ (KHÔNG CẦN SỬA)
# # ============================================================================

# def calculate_3d_angle(a, b, c):
#     """Tính góc gập giữa 3 điểm trong không gian 3D"""
#     ba = a - b
#     bc = c - b
#     cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
#     angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
#     return angle

# def get_elevation_azimuth(p_start, p_end):
#     """
#     Tính góc nâng (Elevation - Pitch) và góc ngang (Azimuth - Yaw)
#     của một vector trong hệ tọa độ chuẩn (MediaPipe)
#     """
#     dx = p_end[0] - p_start[0]
#     dy = p_end[1] - p_start[1]
#     dz = p_end[2] - p_start[2]

#     # MediaPipe: Y xuống (+), X phải (+), Z tới (+)
#     # Chuyển sang hệ quy chiếu người quan sát để dễ hình dung:
#     # Up/Down (Pitch): Dựa vào DY
#     # Left/Right (Yaw): Dựa vào DX
#     # Front/Back: Dựa vào DZ

#     # Tính Pitch (Nâng hạ): Góc với mặt phẳng ngang (XZ)
#     # distance_horizontal = sqrt(dx*dx + dz*dz)
#     # pitch = degrees(atan2(dy, distance_horizontal)) # -90 (lên) đến 90 (xuống)

#     # Ở đây ta dùng vector đơn vị để map vào cấu hình
#     v = np.array([dx, dy, dz])
#     v = v / (np.linalg.norm(v) + 1e-6)

#     # Góc Pitch vật lý (Đưa tay ra trước/sau)
#     # Z âm là đưa ra trước (trong MP Z+ là xa camera, Z- là gần camera?)
#     # MP: Z value is depth. Negative is closer to camera.
#     # Vector cánh tay đưa ra trước: dz sẽ âm lớn.
#     pitch_physical = np.degrees(np.arctan2(v[2], v[1])) # Góc trong mặt phẳng YZ

#     # Góc Yaw vật lý (Dang tay sang ngang)
#     yaw_physical = np.degrees(np.arctan2(v[0], v[1]))   # Góc trong mặt phẳng XY

#     return pitch_physical, yaw_physical

# # ============================================================================
# # KINEMATIC SOLVER
# # ============================================================================

# class KinematicSolver:
#     def __init__(self):
#         pass

#     def apply_config(self, phys_val, config_key, part_config):
#         """Map giá trị vật lý vào trục model dựa trên RIG_CONFIG"""
#         cfg = part_config[config_key]
#         axis = cfg['axis']
#         scale = cfg['scale']
#         offset = cfg['offset']

#         mapped_val = (phys_val * scale) + offset
#         return axis, int(mapped_val)

#     def create_rot_string(self, name, mapping):
#         """Tạo chuỗi rotation từ dict mapping {axis: value}"""
#         x = mapping.get('x', 0)
#         y = mapping.get('y', 0)
#         z = mapping.get('z', 0)
#         return f"{name}(x={x}, y={y}, z={z})"

#     def solve_arm(self, landmarks, side='left'):
#         # 1. Lấy tọa độ landmarks
#         if side == 'left':
#             sh = np.array([landmarks[11].x, landmarks[11].y, landmarks[11].z])
#             el = np.array([landmarks[13].x, landmarks[13].y, landmarks[13].z])
#             wr = np.array([landmarks[15].x, landmarks[15].y, landmarks[15].z])
#             cfg_name = "left_arm"
#             fore_name = "left_forearm"
#             suffix = 'l'
#         else:
#             sh = np.array([landmarks[12].x, landmarks[12].y, landmarks[12].z])
#             el = np.array([landmarks[14].x, landmarks[14].y, landmarks[14].z])
#             wr = np.array([landmarks[16].x, landmarks[16].y, landmarks[16].z])
#             cfg_name = "right_arm"
#             fore_name = "right_forearm"
#             suffix = 'r'

#         # 2. Tính toán vật lý (Physical Calculation)

#         # A. Bắp tay (Upper Arm)
#         # Vector vai -> khuỷu
#         vec_upper = el - sh

#         # Góc đưa trước/sau (Pitch): Dựa trên trục Z của MP
#         # Khi tay xuôi (Y+), đưa ra trước (Z-).
#         # Ta cần tính độ mở góc so với trục dọc cơ thể.
#         # atan2(z, y): 0 là xuôi, -90 là đưa thẳng ra trước.
#         pitch_phys = np.degrees(np.arctan2(vec_upper[2], vec_upper[1]))

#         # Góc dang ngang (Yaw): Dựa trên trục X của MP
#         # atan2(x, y): 0 là xuôi, 90 là dang ngang.
#         yaw_phys = np.degrees(np.arctan2(vec_upper[0], vec_upper[1]))

#         # B. Cẳng tay (Forearm)
#         # Góc gập khuỷu tay (luôn dương, 180 là thẳng, 90 là vuông)
#         elbow_angle = calculate_3d_angle(sh, el, wr)
#         # Model thường tính góc gập từ 0 (thẳng) đến 90 (vuông) hoặc ngược lại
#         # Giả sử model của bạn: 0 là thẳng.
#         bend_phys = 180 - elbow_angle

#         # 3. Mapping vào Model (Dùng RIG_CONFIG)
#         u_map = {'x': 0, 'y': 0, 'z': 0}
#         l_map = {'x': 0, 'y': 0, 'z': 0}

#         # Map Upper Arm
#         p_axis, p_val = self.apply_config(pitch_phys, 'pitch', RIG_CONFIG[cfg_name])
#         y_axis, y_val = self.apply_config(yaw_phys, 'yaw', RIG_CONFIG[cfg_name])

#         # Cộng dồn giá trị vào trục (vì 1 trục có thể chịu tác động của cả pitch và yaw tùy model)
#         u_map[p_axis] += p_val
#         u_map[y_axis] += y_val

#         # Map Lower Arm
#         b_axis, b_val = self.apply_config(bend_phys, 'bend', RIG_CONFIG[fore_name])
#         l_map[b_axis] += b_val

#         # 4. Tạo output string
#         upper_str = self.create_rot_string(f"upperarm_{suffix}", u_map)
#         lower_str = self.create_rot_string(f"lowerarm_{suffix}", l_map)

#         return {
#             f"upperarm_{suffix}": upper_str,
#             f"lowerarm_{suffix}": lower_str,
#             f"shoulder_{suffix}": f"shoulder_{suffix}(x=0,y=0,z=0)", # Vai thường ít động
#             f"hand_{suffix}": f"hand_{suffix}(x=0,y=0,z=0)" # Tạm thời để 0
#         }

#     def solve_fingers(self, landmarks, side='left'):
#         # Logic gập ngón tay (tính góc gập và map vào trục cấu hình)
#         suffix = 'l' if side == 'left' else 'r'
#         fingers_out = {}

#         f_indices = {
#             'thumb': [1, 2, 3, 4], 'index': [5, 6, 7, 8],
#             'middle': [9, 10, 11, 12], 'ring': [13, 14, 15, 16],
#             'pinky': [17, 18, 19, 20]
#         }

#         cfg = RIG_CONFIG['fingers']
#         axis = cfg['bend_axis']
#         scale = cfg['bend_scale']

#         for name, ids in f_indices.items():
#             # Lấy landmarks
#             # Hand landmarks coordinates are relative to wrist usually in MP Hand model,
#             # but here we have holistic.

#             # Tính góc gập trung bình
#             # MCP bend
#             angle = calculate_3d_angle(
#                 np.array([landmarks[ids[0]].x, landmarks[ids[0]].y, landmarks[ids[0]].z]),
#                 np.array([landmarks[ids[1]].x, landmarks[ids[1]].y, landmarks[ids[1]].z]),
#                 np.array([landmarks[ids[2]].x, landmarks[ids[2]].y, landmarks[ids[2]].z])
#             )

#             # 180 là thẳng, 90 là gập.
#             # Model: 0 là thẳng, 90 là gập (thường vậy)
#             bend_val = int((180 - angle) * scale)

#             # Tạo chuỗi cho 3 đốt (giả sử gập đều)
#             # Bone names: name_01, name_02, name_03
#             f_str = ""
#             for i in range(1, 4):
#                 # Ngón cái thường chỉ có 2 đốt xoay chính hoặc đặt tên khác, nhưng theo format cũ của bạn:
#                 # thumb_01, thumb_02, thumb_03
#                 # Với ngón cái, đốt 1 (CMC) ít gập hơn
#                 val = bend_val
#                 if name == 'thumb' and i == 1: val = int(bend_val * 0.5)

#                 f_map = {'x': 0, 'y': 0, 'z': 0}
#                 f_map[axis] = val

#                 seg_str = self.create_rot_string(f"{name}_0{i}_{suffix}", f_map)
#                 f_str += seg_str + "; "

#             fingers_out[name] = f_str.strip("; ")

#         return fingers_out

# # ============================================================================
# # MAIN DETECTOR CLASS
# # ============================================================================

# class SignLanguageDetectorV6:
#     def __init__(self):
#         self.mp_holistic = mp.solutions.holistic
#         self.holistic = self.mp_holistic.Holistic(
#             min_detection_confidence=0.5,
#             min_tracking_confidence=0.5,
#             model_complexity=1
#         )
#         self.solver = KinematicSolver()
#         print("✅ SignLanguageDetector V6 (Custom Rig) initialized")

#     def process_frame(self, frame) -> Optional[Dict]:
#         img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         results = self.holistic.process(img_rgb)

#         if not results.pose_landmarks: return None
#         pose_lm = results.pose_landmarks.landmark

#         # Analyze Arms
#         left_arm = self.solver.solve_arm(pose_lm, 'left')
#         right_arm = self.solver.solve_arm(pose_lm, 'right')

#         # Analyze Fingers
#         left_fingers = {}
#         right_fingers = {}

#         if results.right_hand_landmarks: # MP right is person left
#             left_fingers = self.solver.solve_fingers(results.right_hand_landmarks.landmark, 'left')

#         if results.left_hand_landmarks:
#             right_fingers = self.solver.solve_fingers(results.left_hand_landmarks.landmark, 'right')

#         # Combine Output
#         output = {
#             "spine": {"spine_01": "spine_01(x=0,y=0,z=0)"}, # Default
#             "facial": {}, # Default
#             **left_arm,
#             **right_arm,
#             "left_hand_fingers": left_fingers, # Structure slightly adjusted for clarity, you can flatten if needed
#             "right_hand_fingers": right_fingers
#         }

#         # Merge fingers back to arm dict structure if needed by Android app
#         # (Assuming Android app expects fingers inside left_arm dict)
#         output["left_arm"]["fingers"] = left_fingers
#         output["right_arm"]["fingers"] = right_fingers

#         # Clean up flat keys if structure requires nesting
#         # Based on previous JSON: left_arm contains fingers.

#         final_output = {
#             "spine": {"spine_01": "spine_01(x=0,y=0,z=0)"},
#             "facial": {"jaw": "jaw(z=0)"}, # Minimal defaults
#             "left_arm": {
#                 **left_arm,
#                 "fingers": left_fingers
#             },
#             "right_arm": {
#                 **right_arm,
#                 "fingers": right_fingers
#             }
#         }

#         return final_output

#     def close(self):
#         self.holistic.close()

# # ============================================================================
# # RUNNER
# # ============================================================================

# def process_video(video_path, output_json, frames_per_minute=60):
#     detector = SignLanguageDetectorV6()
#     cap = cv2.VideoCapture(video_path)

#     if not cap.isOpened():
#         print("Error opening video")
#         return

#     fps = cap.get(cv2.CAP_PROP_FPS)
#     frame_interval = int((fps * 60) / frames_per_minute) if frames_per_minute > 0 else 1

#     results = []
#     frame_id = 0

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret: break

#         if frame_id % frame_interval == 0:
#             data = detector.process_frame(frame)
#             if data:
#                 results.append({
#                     "frame": frame_id,
#                     "timestamp": round(frame_id/fps, 3),
#                     "gestures": data
#                 })
#                 if frame_id % 100 == 0: print(f"Processed frame {frame_id}")

#         frame_id += 1

#     cap.release()
#     detector.close()

#     with open(output_json, 'w') as f:
#         json.dump(results, f, indent=2)
#     print(f"Done. Saved to {output_json}")

# if __name__ == "__main__":
#     VIDEO = "test_video.webm"
#     if os.path.exists(VIDEO):
#         process_video(VIDEO, "gestures_v6_custom.json")
#     else:
#         print("Video not found")